In [26]:
# cell 1
!pip install streamlit pyngrok ultralytics opencv-python-headless
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install cython_bbox

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-9_l_04c3
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-9_l_04c3
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done


In [27]:
# cell 2
%%writefile app.py
import streamlit as st
import os
import tempfile
import json

try:
    from video_tracker import track_video
    TRACKER_AVAILABLE = True
except ImportError as e:
    st.error(f"Error importing video_tracker: {e}")
    TRACKER_AVAILABLE = False

st.set_page_config(
    page_title="Vehicle & Pedestrian Tracker",
    layout="wide"
)

st.title("(EXTRA SUBMISSION) Second Web app sample to show how yolo tracks")
st.markdown("Upload a video to track vehicles and pedestrians using YOLOv8 with ByteTrack tracking")

MODEL_WEIGHTS_PATH = "best.pt"

if not TRACKER_AVAILABLE:
    st.error("Tracking functionality not available. Please check video_tracker.py file.")
elif not os.path.exists(MODEL_WEIGHTS_PATH):
    st.error(f"Model weights file not found at '{MODEL_WEIGHTS_PATH}'")
else:
    st.success(f"Model found: {MODEL_WEIGHTS_PATH}")

    uploaded_file = st.file_uploader("Upload a video file", type=["mp4", "mov", "avi", "mkv"])

    if uploaded_file is not None:
        file_size = uploaded_file.size / (1024 * 1024)  # Convert to MB
        st.info(f"Uploaded video: {uploaded_file.name} ({file_size:.2f} MB)")

        with tempfile.NamedTemporaryFile(delete=False, suffix='.mp4') as tfile:
            tfile.write(uploaded_file.getbuffer())
            input_video_path = tfile.name

        st.subheader("Original Video")
        st.video(input_video_path)

        if st.button("Start Tracking", type="primary"):
            output_video_path = f"tracked_{uploaded_file.name}"
            results_json_path = "tracking_results.json"

            progress_bar = st.progress(0)
            status_text = st.empty()

            with st.spinner("Processing video... This may take a few minutes depending on video length.You can donload .json or output video.If you want both re run the app"):
                success, message = track_video(
                    input_video_path,
                    output_video_path,
                    MODEL_WEIGHTS_PATH,
                    results_json_path
                )

                if success:
                    progress_bar.progress(100)
                    status_text.success("Processing Complete!")

                    st.success("Tracking completed successfully!")

                    st.subheader("Tracked Video")
                    if os.path.exists(output_video_path):
                        st.video(output_video_path)
                    else:
                        st.error("Output video file not found")

                    st.markdown("---")
                    st.subheader("Download Results")

                    col1, col2 = st.columns(2)

                    with col1:
                        if os.path.exists(results_json_path):
                            with open(results_json_path, "rb") as f:
                                st.download_button(
                                    label="Download Tracking Results (JSON)",
                                    data=f,
                                    file_name="tracking_results.json",
                                    mime="application/json",
                                    help="Contains frame-by-frame tracking data"
                                )

                    with col2:
                        if os.path.exists(output_video_path):
                            with open(output_video_path, "rb") as f:
                                st.download_button(
                                    label="Download Tracked Video",
                                    data=f,
                                    file_name=output_video_path,
                                    mime="video/mp4",
                                    help="Video with bounding boxes and tracking IDs"
                                )

                    try:
                        os.unlink(input_video_path)
                        if os.path.exists(output_video_path):
                            os.remove(output_video_path)
                        if os.path.exists(results_json_path):
                            os.remove(results_json_path)
                    except Exception as e:
                        st.warning(f"Could not clean up temporary files: {e}")

                else:
                    st.error(f"Processing failed: {message}")

                try:
                    os.unlink(input_video_path)
                except:
                    pass

Overwriting app.py


In [28]:
# cell 3
%%writefile video_tracker.py
import cv2
import json
from ultralytics import YOLO
import numpy as np

def track_video(input_path, output_path, model_path, results_json_path):
    """
    Track objects in video using YOLOv8 and return tracking results
    """
    try:
        # Load YOLOv8 model
        model = YOLO(model_path)

        # Open video
        cap = cv2.VideoCapture(input_path)

        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Initialize video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        tracking_results = []
        frame_count = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Run tracking on frame
            # Using YOLOv8's built-in tracker
            results = model.track(frame, persist=True, verbose=False)

            frame_data = {
                'frame': frame_count,
                'objects': []
            }

            # Process results
            if results[0].boxes is not None and results[0].boxes.id is not None:
                boxes = results[0].boxes.xyxy.cpu().numpy()
                track_ids = results[0].boxes.id.cpu().numpy()
                confidences = results[0].boxes.conf.cpu().numpy()
                classes = results[0].boxes.cls.cpu().numpy()

                for i, (box, track_id, conf, cls) in enumerate(zip(boxes, track_ids, confidences, classes)):
                    x1, y1, x2, y2 = box

                    # Store object data
                    obj_data = {
                        'id': int(track_id),
                        'class': model.names[int(cls)],
                        'confidence': float(conf),
                        'bbox': [float(x1), float(y1), float(x2), float(y2)]
                    }
                    frame_data['objects'].append(obj_data)

                    # Draw bounding box and ID on frame
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                    cv2.putText(frame, f'ID: {int(track_id)} {model.names[int(cls)]}',
                               (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            tracking_results.append(frame_data)
            out.write(frame)
            frame_count += 1

        # Cleanup
        cap.release()
        out.release()

        # Save tracking results
        results_data = {
            'video_info': {
                'total_frames': total_frames,
                'fps': fps,
                'width': width,
                'height': height
            },
            'tracking_results': tracking_results
        }

        with open(results_json_path, 'w') as f:
            json.dump(results_data, f, indent=2)

        return True, "Tracking completed successfully"

    except Exception as e:
        return False, str(e)

Overwriting video_tracker.py


In [29]:
# cell 4
# Download YOLOv8 model weights (you can use a pre-trained or custom model)
from ultralytics import YOLO

# Download a pre-trained YOLOv8 model
model = YOLO('yolov8n.pt')  # or yolov8s.pt, yolov8m.pt, yolov8l.pt, yolov8x.pt

# Save as best.pt (as expected by the app)
model.save('best.pt')
print("Model downloaded and saved as 'best.pt'")

Model downloaded and saved as 'best.pt'


In [30]:
# from pyngrok import ngrok, conf
# ngrok.kill()  # kills all running tunnels


In [32]:
# cell 5
import subprocess
import threading
import time
from pyngrok import ngrok

# Set your ngrok auth token (get it from https://dashboard.ngrok.com/get-started/your-authtoken)
ngrok.set_auth_token("336k5IkJjdrp2jqviB8jHr6eNtr_4rJFLd1ZohXQLjRFYDRUp")


def run_streamlit():
    subprocess.run(["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"])

thread = threading.Thread(target=run_streamlit)
thread.daemon = True
thread.start()


time.sleep(10)


public_url = ngrok.connect(8501)
print(f"Your Streamlit app is running at: {public_url}")
print("Click the link above to access your application!")

Your Streamlit app is running at: NgrokTunnel: "https://endodermal-overpotently-myrle.ngrok-free.dev" -> "http://localhost:8501"
Click the link above to access your application!
